In [1]:
import tensorflow as tf

In [21]:
# load the data

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data() # the data is in the form of two sequences

In [22]:
# reshape the images

x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
x_train, x_test = x_train / 255.0, x_test / 255.0    

In [23]:
# creating a custom callback class to stop training after achieve 99.8% accuracy

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') >= 0.998):
            print("Reached 99.8% accuracy so cancelling training!")
            self.model.stop_training = True
            
callbacks = myCallback()

In [24]:
# create the model using the Sequential API

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [25]:
# compiling the model

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [26]:
# fitting the model

model.fit(x_train, y_train,
         epochs=20, batch_size=128,
         callbacks=[callbacks])

Train on 60000 samples
Epoch 1/20
60000/60000 [==============================] - 26s 426us/sample - loss: 0.2241 - accuracy: 0.9363
Epoch 2/20
60000/60000 [==============================] - 25s 415us/sample - loss: 0.0695 - accuracy: 0.9796
Epoch 3/20
60000/60000 [==============================] - 25s 411us/sample - loss: 0.0468 - accuracy: 0.9858
Epoch 4/20
60000/60000 [==============================] - 25s 412us/sample - loss: 0.0355 - accuracy: 0.9891
Epoch 5/20
60000/60000 [==============================] - 25s 415us/sample - loss: 0.0271 - accuracy: 0.9915
Epoch 6/20
60000/60000 [==============================] - 25s 414us/sample - loss: 0.0205 - accuracy: 0.9938
Epoch 7/20
60000/60000 [==============================] - 25s 420us/sample - loss: 0.0155 - accuracy: 0.9953
Epoch 8/20
60000/60000 [==============================] - 25s 419us/sample - loss: 0.0120 - accuracy: 0.9967
Epoch 9/20
60000/60000 [==============================] - 25s 425us/sample - loss: 0.0092 - accuracy: 0.9